In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pickle
import numpy as np

In [3]:
df = pd.read_csv('/home/nikolay/EMAI/Ljublajna/Explainable-AI-Project/user_study_dataset/models/data.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1,55.0,0,0,1,45.20,9.0,280,1
1,0,14.0,0,0,0,20.13,6.1,126,0
2,1,64.0,1,0,0,33.87,9.0,140,1
3,0,72.0,0,0,0,22.79,7.0,140,1
4,0,36.0,0,0,0,40.00,5.7,260,1


In [6]:
df = pd.get_dummies(df, drop_first=True)

In [7]:
rf = RandomForestClassifier(n_estimators=30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df.drop('diabetes', axis=1), df['diabetes'], test_size=0.2, random_state=42)

rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=30, random_state=42)

In [8]:
preds = rf.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, preds))

Accuracy:  0.8850488354620586


In [19]:
cf_sample = X_test.iloc[0]

In [33]:
cf_sample_df = pd.DataFrame(cf_sample).transpose()

In [40]:
cf_sample_df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
6113,1.0,54.0,0.0,0.0,1.0,35.02,7.0,160.0


In [34]:
cf_sample_df.shape

(1, 8)

In [51]:
from carla import MLModelCatalog
from carla.data.catalog import OnlineCatalog, CsvCatalog
from carla.recourse_methods import GrowingSpheres

# load a catalog dataset
data_name = "compas"
dataset = OnlineCatalog(data_name)
#dataset = CsvCatalog('/home/nikolay/EMAI/Ljublajna/Explainable-AI-Project/user_study_dataset/models/data.csv',
#                      categorical= ['gender', 'hypertension', 'heart_disease', 'smoking_history'],
#                      continuous = ['age', 'bmi', 'blood_glucose_level', 'HbA1c_level'],
#                      immutables= ['age'],
#                      target = 'diabetes')

# load artificial neural network from catalog
model = MLModelCatalog(dataset, model_type='ann')
#MLModelCatalog

In [3]:
dataset.df

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.301370,0.044131,0.800000,0.021740,0.0,...,0.0,1.0,1.0,1.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.000000,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.000000,0.0,...,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
48827,0.301370,0.137428,0.800000,0.000000,0.0,...,0.0,1.0,1.0,0.0,1.0
48828,0.643836,0.209130,0.533333,0.000000,0.0,...,0.0,0.0,1.0,1.0,1.0
48829,0.287671,0.245379,0.800000,0.000000,0.0,...,0.0,1.0,0.0,1.0,1.0
48830,0.369863,0.048444,0.800000,0.054551,0.0,...,0.0,0.0,1.0,1.0,1.0


In [32]:
# get factuals from the data to generate counterfactual examples
factuals = dataset.df[:1]

# load a recourse model and pass black box model
gs = GrowingSpheres(model)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factuals)

/home/nikolay/EMAI/Ljublajna/Explainable-AI-Project/.venv/lib/python3.7/site-packages/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  f"cannot re-order features for non dataframe input: {type(x)}"


In [20]:
factuals['income']

0    0
Name: income, dtype: int64

In [34]:
counterfactuals.to_json(orient='records')

'[{"age":0.301369863,"fnlwgt":0.035789848,"education-num":0.8874855822,"capital-gain":0.0470831126,"capital-loss":0.0319958191,"hours-per-week":0.3979756331,"workclass_Private":0.0,"marital-status_Non-Married":1.0,"occupation_Other":0.0,"relationship_Non-Husband":1.0,"race_White":1.0,"sex_Male":1.0,"native-country_US":1.0}]'

In [18]:
for i in counterfactuals.columns:
    if counterfactuals[i][0] != factuals[i][0]:
        print(f'{i} column original {counterfactuals[i][0]}, new {factuals[i][0]}')

fnlwgt column original 0.03778665713784712, new 0.04413120765299046
education-num column original 0.8207512736473496, new 0.8
capital-gain column original 0.1088859872377777, new 0.021740217402174022
capital-loss column original -0.022802639473581427, new 0.0
hours-per-week column original 0.30986446295645353, new 0.39795918367346933


In [46]:
model.predict(factuals)

array([[0.3001735]], dtype=float32)

In [49]:
model.predict(counterfactuals)[0][0]

0.6038832

In [23]:
def get_column_values(df, col):
    if sorted(df[col].unique().tolist()) == [0, 1]:
        return {'values': df[col].map({0: False, 1: True}).unique().tolist()}
    elif df[col].nunique() < 12:
        return {'values': df[col].unique().tolist()}
    else:
        return {}

def get_column_metadata(df):
    return {
            col: {
                'type': "categorical" if df[col].nunique() < 12 else "numeric",
                **(get_column_values(df, col))
            } for col in df.columns
        }


get_column_metadata(dataset.df)

{'age': {'type': 'numeric'},
 'fnlwgt': {'type': 'numeric'},
 'education-num': {'type': 'numeric'},
 'capital-gain': {'type': 'numeric'},
 'capital-loss': {'type': 'numeric'},
 'hours-per-week': {'type': 'numeric'},
 'income': {'type': 'categorical', 'values': [False, True]},
 'marital-status_Non-Married': {'type': 'categorical',
  'values': [True, False]},
 'native-country_US': {'type': 'categorical', 'values': [True, False]},
 'occupation_Other': {'type': 'categorical', 'values': [False, True]},
 'race_White': {'type': 'categorical', 'values': [True, False]},
 'relationship_Non-Husband': {'type': 'categorical', 'values': [True, False]},
 'sex_Male': {'type': 'categorical', 'values': [True, False]},
 'workclass_Private': {'type': 'categorical', 'values': [False, True]}}

In [52]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset()

In [55]:
get_column_metadata(dataset)

{'age': {'type': 'numeric'},
 'workclass': {'type': 'categorical',
  'values': ['Private', 'Self-Employed', 'Other/Unknown', 'Government']},
 'education': {'type': 'categorical',
  'values': ['Bachelors',
   'Assoc',
   'Some-college',
   'School',
   'HS-grad',
   'Masters',
   'Prof-school',
   'Doctorate']},
 'marital_status': {'type': 'categorical',
  'values': ['Single', 'Married', 'Divorced', 'Widowed', 'Separated']},
 'occupation': {'type': 'categorical',
  'values': ['White-Collar',
   'Professional',
   'Service',
   'Blue-Collar',
   'Other/Unknown',
   'Sales']},
 'race': {'type': 'categorical', 'values': ['White', 'Other']},
 'gender': {'type': 'categorical', 'values': ['Female', 'Male']},
 'hours_per_week': {'type': 'numeric'},
 'income': {'type': 'categorical', 'values': [False, True]}}